In [27]:
# Dependencies and Setup
import pandas as pd

# File to Load 
file_path = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_df = pd.read_csv(file_path)

In [28]:
#Player Count

# Calculate the total number of players by identifying the unique players
t_players = len(purchase_df["SN"].unique()) 

#Create the data frame to display results
t_players_df = pd.DataFrame({"Total Players" : [t_players] })

#Show results
t_players_df

,Total Players
0,576


In [29]:
#Purchasing Analysis (Total)

# Calculate the total number of purchases
t_purchases = len(purchase_df["SN"])

#Identifying the count of unique items
unique_items = len(purchase_df["Item ID"].unique())

#Calculate the average price and giving it $ format
av_price =  "${:,.2f}".format(purchase_df["Price"].sum()/t_purchases)

#Calculate the total revenue and giving it $ format
t_revenue = "${:,.2f}".format(purchase_df["Price"].sum())

#Create the data frame to display results
purchasing_analysis = pd.DataFrame({"Number of Unique Items" : [unique_items], "Average Price": [av_price], 
                                    "Number of Purchases" : [t_purchases], "Total Revenue" : [t_revenue]})
#Show results
purchasing_analysis


,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


In [30]:
# Gender Demographics

#Drop duplicate players
purchase_unique = purchase_df.drop_duplicates(subset=["SN"])

#Count the unique players by gender
gender_counts = purchase_unique["Gender"].value_counts()

#Calculate the percentage of players by gender
per_players = (gender_counts/t_players)*100

#Create the data frame to display results
gender_dem = pd.DataFrame({"Total Count" : gender_counts, "Percentage of Players" : per_players})

#Show results with a cleaner format
gender_dem.style.format({"Percentage of Players" : "{:,.2f}%"})


,Total Count,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [31]:
#Purchasing Analysis (Gender)

#Group the data frame by gender and make some calculations depending on the variables
gender_purchase = purchase_df.groupby("Gender").agg({"Purchase ID":"count", "Price": ["mean", "sum"],
                                                         "SN" : pd.Series.nunique})

#Calculate the Average Total Purchase per Person
av_pp = gender_purchase[("Price","sum")] / gender_purchase[("SN", "nunique")] 

#Asign the Average Total Purchase per Person to the SN nunique column of the groupby dataframe
gender_purchase[(                           'SN', 'nunique')] = av_pp

#Dictionary of the new name of the columns
cols_gender = {('Purchase ID', 'count') : "Purchase Count",
            ('Price', 'mean') : "Average Purchase Price",
            ('Price', 'sum') : "Total Purchase Value",
            ('SN', 'nunique') : "Avg Total Purchase per Person"}

#Convert MultiIndex to SingleIndex
gender_purchase.columns = gender_purchase.columns.to_flat_index()

#Rename the columns of the data frame
rename_gender = gender_purchase.rename(columns=cols_gender)

#Show results with a cleaner format
rename_gender.style.format({"Average Purchase Price" : "${:,.2f}",
                            "Total Purchase Value" : "${:,.2f}",
                            "Avg Total Purchase per Person" : "${:,.2f}" })


,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [32]:
#Age Demographics

#Determine the bins wanted
bins = [0,9.9,14,19,24,29,34,39,purchase_df["Age"].max()]

#Define bin names
group_names = ["<10", "10-14", "15-19", "20-24", "25-29","30-34", "35-39", "40+"]

#Create the bins and add them to the original data frame
purchase_df["Age Ranges"] = pd.cut(purchase_df["Age"], bins, labels = group_names, include_lowest=True)

#Drop duplicate players
ages_unique = purchase_df.drop_duplicates(subset=["SN"])

#Count by age ranges and specify that we don't want to sort the values in order to preserve the original order
age_counts = ages_unique["Age Ranges"].value_counts(sort=False)

#Calculate the percentage of player by age bin
per_age = (age_counts/t_players)*100

#Create the data frame to display results
age_dem = pd.DataFrame({"Total Count" : age_counts, "Percentage of Players" : per_age})

#Show results with a cleaner format
age_dem.style.format({"Percentage of Players" : "{:,.2f}%"})


,Total Count,Percentage of Players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [33]:
#Purchasing Analysis (Age)

#Group the data frame by age ranges and make some calculations depending on the variables
age_purchase = purchase_df.groupby("Age Ranges").agg({"Purchase ID":"count", "Price": ["mean", "sum"],
                                                         "SN" : pd.Series.nunique})
#Calculate the Average Total Purchase per Person
av_pp_age = age_purchase[("Price","sum")] / age_purchase[("SN", "nunique")] 

#Asign the Average Total Purchase per Person to the SN nunique column of the groupby dataframe
age_purchase[(                           'SN', 'nunique')] = av_pp_age

#Dictionary of the new name of the columns
cols_age = {('Purchase ID',   'count') : "Purchase Count",
            ('Price',    'mean') : "Average Purchase Price",
            ('Price',     'sum') : "Total Purchase Value",
            ('SN', 'nunique') : "Avg Total Purchase per Person"}

#Convert MultiIndex to SingleIndex
age_purchase.columns = age_purchase.columns.to_flat_index()

#Rename the columns of the data frame
rename_age = age_purchase.rename(columns=cols_age)

#Show results with a cleaner format
rename_age.style.format({"Average Purchase Price" : "${:,.2f}",
                            "Total Purchase Value" : "${:,.2f}",
                            "Avg Total Purchase per Person" : "${:,.2f}" })


,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Ranges,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [34]:
#Top Spenders

#Group the data frame by SN and make some calculations depending on the variables
top_spenders =purchase_df.groupby("SN").agg({"Purchase ID":"count", "Price": ["mean", "sum"]})

#Dictionary of the new name of the columns
cols_spenders= {('Purchase ID',   'count') : "Purchase Count",
            ('Price',    'mean') : "Average Purchase Price",
            ('Price',     'sum') : "Total Purchase Value"}

#Convert MultiIndex to SingleIndex
top_spenders.columns = top_spenders.columns.to_flat_index()

#Rename the columns of the data frame
rename_spenders = top_spenders.rename(columns=cols_spenders)

#Sort values by Purchase Count
spenders_sort1 = rename_spenders.sort_values("Purchase Count", ascending=False)

#Sort again but now by Totaal Purchase Value
spenders_sort2 = spenders_sort1.sort_values("Total Purchase Value", ascending=False)

#Get the data of the 5 top spenders
summary_spenders = spenders_sort2.head(5)

#Show results with a cleaner format
summary_spenders.style.format({"Average Purchase Price" : "${:,.2f}",
                            "Total Purchase Value" : "${:,.2f}"})


,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [35]:
#Most Popular Items

#Group the data frame by Item ID and by Item Name and make some calculations depending on the variables
top_items =purchase_df.groupby(["Item ID","Item Name"]).agg({"Purchase ID":"count", "Price": ["mean", "sum"]})

#Dictionary of the new name of the columns
cols_items= {('Purchase ID',   'count') : "Purchase Count",
            ('Price',    'mean') : "Item Price",
            ('Price',     'sum') : "Total Purchase Value"}

#Convert MultiIndex to SingleIndex
top_items.columns = top_items.columns.to_flat_index()

#Rename the columns of the data frame
rename_items = top_items.rename(columns=cols_items)

#Sort values by Purchase Count
items_sort1 = rename_items.sort_values("Purchase Count", ascending=False)

#Get the data of the 5 top spenders
summary_items = items_sort1.head(5)

#Show results with a cleaner format
summary_items.style.format({"Item Price" : "${:,.2f}",
                            "Total Purchase Value" : "${:,.2f}"})



,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


In [36]:
#Most Profitable Items


#Sort values by Total Purchase Value
items_sort2 = items_sort1.sort_values("Total Purchase Value", ascending = False)

#Get the data of the 5 top spenders
summary_items2 = items_sort2.head(5)

#Show results with a cleaner format
summary_items2.style.format({"Item Price" : "${:,.2f}",
                            "Total Purchase Value" : "${:,.2f}"})



,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
